In [ ]:
from convinience import Spectrum, Band, synthesisSpectrum
from convinience import gaussian_profile, compensation, calc_totalPower ,attenuation
from WSMethods import uploadProfile, getWSrange
import json
import copy
from scipy import interpolate
import traceback
from matplotlib import pyplot as plt
import numpy as np
import ipywidgets as widgets
import datetime
import os

from IPython.display import display
from ipyfilechooser import FileChooser
from time import sleep
try: import thorlabs_apt as apt
except Exception as e: print(e)
import pyvisa
from scipy.optimize import minimize

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default='notebook' # for vscode ,maybe 'colab' on jupyterlab 

In [ ]:
# 各種設定周り ここにすべてまとめる

class Setting:
    def __init__(self):
        global hd_idx
        self.ws_ip                          = '169.254.6.8'
        self.ws_Freq_resolution             = 0.001 # 0.001THz = 1GHz
        self.ws_Freqrange                   = None
        self.optSpectrum : Spectrum         = None
        self.mso58_address                  = 'USB::0x0699::0x0530::C043144::INSTR'
        self.homodyne_port                  = 1
        self.fastFramePulseNum              = 5
        #self.quadrature_method              = lambda volts: PCA().fit_transform(volts.reshape((-1,volts.shape[-1])))[:,0].reshape(volts.shape[:-1])
        self.quadrature_method              = lambda volts: volts.reshape(-1,volts.shape[-1])[:,hd_idx].reshape(volts.shape[:-1])
        self.n9010b_address                 = 'TCPIP0::K-N9010B-42308.local::hislip0::INSTR'
        self.hd_trace                       = 1
        self.speana_idx_from                = 500
        self.speana_idx_to                  = 1700
        self.targetLOPower                  = 1
        self.shutter_port                   = 'COM7'
        self.pumpShutterPort : int          = 2
        self.param_csv                      = f"{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}record.csv"


    def condition_approval(self):
        assert self.ws_Freqrange             is not None
        assert self.optSpectrum              is not None
        assert self.optSpectrum.powerdensity is not None
        assert self.optSpectrum.frequency    is not None
        assert self.optSpectrum.wavelength   is not None
        assert self.optSpectrum.phase        is not None
        assert self.fastFramePulseNum        is not None
        assert self.quadrature_method        is not None
        assert self.targetLOPower            is not None
        assert self.shutter_port             is not None
        assert self.pumpShutterPort          is not None
        assert self.param_csv                is not None


setting = Setting()

startFreq, endFreq    = 191.05, 196.475# getWSrange(setting.ws_ip)
setting.ws_Freqrange  = np.arange(startFreq, endFreq, setting.ws_Freq_resolution)

setting.optSpectrum                = Spectrum()
setting.optSpectrum.band           = Band()
setting.optSpectrum.frequency      = np.load('optFreq.npy')
setting.optSpectrum.powerdensity   = np.load('optSpectrum.npy')
setting.optSpectrum.phase          = np.zeros_like(setting.optSpectrum.frequency)

setting.condition_approval()

In [ ]:
class State:
    def __init__(self):
        self.current_spectrum = None
        self.former_spectrum  = None
        self.former_diff = 0
        self.diff_history = []

state = State()

In [ ]:
def apt_device_list():
    print(apt.list_available_devices())

def stage_move_to():
    global stage_serial_num
    global stage_target_pos
    motor = apt.Motor(stage_serial_num)
    motor.move_to(stage_target_pos, blocking=True)

def stage_velocity():
    global stage_serial_num
    global stage_max_velocity
    motor = apt.Motor(stage_serial_num)
    min_v, acc, max_v = motor.get_velocity_parameters()
    motor.set_velocity_parameters(min_v, acc, stage_max_velocity)

def stage_enable():
    global stage_serial_num
    motor = apt.Motor(stage_serial_num)
    motor.enable()

def stage_disable():
    global stage_serial_num
    motor = apt.Motor(stage_serial_num)
    motor.disable()

def start_DMM6500():
    global DMM6500_visa_address
    rm = pyvisa.ResourceManager()
    with rm.open_resource(DMM6500_visa_address) as instr:
        instr.write(':TRACe:CLEar')
        instr.write(':TRIGger:LOAD "LoopUntilEvent", COMMand, 100')
        instr.write('INIT')

def stop_DMM6500():
    global DMM6500_visa_address
    rm = pyvisa.ResourceManager()
    with rm.open_resource(DMM6500_visa_address) as instr:
        instr.write(':ABORT')
    
def read_DMM6500():
    global now
    now = datetime.datetime.now()
    global DMM6500_visa_address
    global x, y
    rm = pyvisa.ResourceManager()
    with rm.open_resource(DMM6500_visa_address) as instr:
        result = instr.query(':TRIGger:STATe?')
        assert 'ABORTED' in result.split(';')
        result = instr.query(':TRACe:ACTual:END?')
        end = int(result)
        assert end > 0
        result = instr.query(f':TRACe:DATA? 1,{end},"defbuffer1",RELative')
        x = np.array(list(map(float,result.split(','))))
        result = instr.query(f':TRACe:DATA? 1,{end},"defbuffer1"')
        y = np.array(list(map(float,result.split(','))))

def plot_xy():
    global x,y
    fig = make_subplots(rows=1, cols=1, subplot_titles=("x,y", "---somethingsomething"))
    go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
    fig.add_trace(go_element(x,y, 'x,y'), row=1, col=1)
    fig.update_xaxes(title_text="x", row=1, col=1)
    fig.update_yaxes(title_text="y", row=1, col=1)
    fig.show()

def save_xy():
    global comment
    global now
    filename_y = now.strftime('%Y%m%d_%H%M%S') + '_' + 'ydata_' + comment   + '.npy'
    filename_x = now.strftime('%Y%m%d_%H%M%S') + '_' +'xdata_' + comment +  '.npy'
    np.save(filename_y,y)
    np.save(filename_x,x)

def measure_autocorrelation():
    global x,y
    global auto_stage_start_pos
    global auto_stage_end_pos
    global stage_target_pos
    global stage_max_velocity
    _store_stage_max_velocity = stage_max_velocity
    stage_max_velocity = 100
    stage_velocity()
    stage_target_pos = auto_stage_start_pos
    stage_move_to()
    stage_max_velocity = _store_stage_max_velocity
    stage_velocity()
    start_DMM6500()
    stage_target_pos = auto_stage_end_pos
    stage_move_to()
    stop_DMM6500()
    read_DMM6500()
    c = 299792458
    x = 2*x*_store_stage_max_velocity*1E-3 / c


def gaussian_fitting():
    global x, y
    global _offset, _amplitude, _t_p, _t_center
    def loglikelihood(param):
        global x,y
        offset = param[0]
        amplitude = param[1]
        t_p = param[2]
        t_center = param[3]
        model_estimation = offset + amplitude*(1.0+2.0*np.exp(-0.5*((x*1E12 - t_center)/t_p)**2))
        diff = y*1000 - model_estimation
        log_like = -np.sum(diff**2)
        return -log_like

    offset = _offset*1000
    amplitude = _amplitude*1000
    t_p = _t_p
    t_center = _t_center
    result = minimize(loglikelihood, [offset,amplitude,t_p,t_center], method='L-BFGS-B')
    _offset = result['x'][0]/1000
    _amplitude = result['x'][1]/1000
    _t_p = result['x'][2]
    _t_center = result['x'][3]


def gaussian_fitting_dialog():
    global _offset, _amplitude, _t_p, _t_center
    button_do_fit = widgets.Button(description='do_fit')
    button_plot = widgets.Button(description='plot')
    float_offset = widgets.FloatText(value=0,description='offset',disabled=False,style={'description_width': 'initial'})
    float_amplitude = widgets.FloatText(value=0,description='amplitude',disabled=False,style={'description_width': 'initial'})
    float_t_p = widgets.FloatText(value=0,description='t_p[ps]',disabled=False,style={'description_width': 'initial'})
    float_t_center = widgets.FloatText(value=0,description='t_center[ps]',disabled=False,style={'description_width': 'initial'})
    try: float_offset.value = _offset
    except Exception: _offset = float_offset.value
    try: float_amplitude.value = _amplitude
    except Exception: _amplitude = float_amplitude.value
    try: float_t_p.value = _t_p
    except Exception: _t_p = float_t_p.value
    try: float_t_center.value = _t_center
    except Exception: _t_center = float_t_center.value

    output = widgets.Output()
    def wrapped_func_factory(func,verbose=True):
        def new_func(ui_element):
            with output:
                if verbose: print(f"[exec func {func.__name__}]")
                func()
                if verbose: print(f"[complete {func.__name__}]")
        return new_func
    
    def do_fit():
        global _offset, _amplitude, _t_p, _t_center
        _offset = float_offset.value
        _amplitude = float_amplitude.value
        _t_p = float_t_p.value
        _t_center = float_t_center.value
        gaussian_fitting()
        float_offset.value = _offset
        float_amplitude.value = _amplitude
        float_t_p.value = _t_p
        float_t_center.value = _t_center

    def plot():
        global x,y
        global _offset, _amplitude, _t_p, _t_center
        _offset = float_offset.value
        _amplitude = float_amplitude.value
        _t_p = float_t_p.value
        _t_center = float_t_center.value
        fig = make_subplots(rows=1, cols=1, subplot_titles=("x,y", "---somethingsomething"))
        go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
        fig.add_trace(go_element(x,y, 'x,y'), row=1, col=1)
        x_range = np.linspace(np.min(x), np.max(x), 300)
        fig.add_trace(go_element(x_range,_offset + _amplitude*(1.0+2.0*np.exp(-0.5*((x_range*1E12 - _t_center)/_t_p)**2)), 'x,estimate'), row=1, col=1)
        fig.update_xaxes(title_text="x", row=1, col=1)
        fig.update_yaxes(title_text="y", row=1, col=1)
        fig.show()

    
    button_do_fit.on_click(wrapped_func_factory(do_fit, verbose=False))
    button_plot.on_click(wrapped_func_factory(plot, verbose=False))

    display(
        widgets.VBox([
            widgets.HBox([button_do_fit,button_plot]),
            widgets.HBox([float_offset, float_amplitude, float_t_p, float_t_center]),
            output
        ])
    )
        

In [ ]:
def playback_bayesian08():
    global setting
    global state
    global _offset, _amplitude, _t_p, _t_center
    global bayes_json
    global now
    global x, y

    with open(bayes_json, "r+") as f:
        counter = 0
        while True:
            now_pos = f.tell()
            try: iteration = next(f).strip('\n')
            except StopIteration: break
            
            json_contents = json.loads(iteration)

            try: 
                json_contents['meta_data']['t_p']
                continue
            except KeyError: pass 
            
            next_probe_point = json_contents['params']
            next_probe_point_trans = {float(k):v for k, v in next_probe_point.items()}
    #        next_probe_point[-3] = 0 # 境界なめらか処理
    #        next_probe_point[3] = 0 # 境界なめらか処理
            # phase generation(next_probe_point)
            band = Band()
            band.frequency          = setting.ws_Freqrange
            filterSpectrum          = gaussian_profile(
                        centerWavelength    =1545.483,
                        bandWidth           =0.5, band=band,
                        dispersion          =-0.9
                    )  
            x = list(next_probe_point_trans.keys())
            x.sort()
            y = [next_probe_point_trans[i] for i in x]
            x = np.array([float(i) for i in x]) * 0.5 + 1545.3
            akima = interpolate.Akima1DInterpolator(x,y)
            filterSpectrum.phase = np.nan_to_num(akima(band.wavelength), 0)
            state.current_spectrum = filterSpectrum # 補償はしていない(^_^;)
            compensated_Spectrum = copy.deepcopy(filterSpectrum)
            
            filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
            r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

            sleep(1)
            measure_autocorrelation()
            gaussian_fitting()
            json_contents['meta_data'] = json_contents.get('meta_data', {})
            json_contents['meta_data']['offset'] = _offset
            json_contents['meta_data']['amplitude'] = _amplitude
            json_contents['meta_data']['t_p'] = _t_p
            json_contents['meta_data']['t_center'] = _t_center
            filename_y = now.strftime('%Y%m%d_%H%M%S') + '_' + 'ydata_' + comment   + '.npy'
            filename_x = now.strftime('%Y%m%d_%H%M%S') + '_' +'xdata_' + comment +  '.npy'
            os.makedirs('autocorrelator_data', exist_ok=True)
            np.save(f"autocorrelator_data/{filename_y}",y)
            np.save(f"autocorrelator_data/{filename_x}",x)
            json_contents['meta_data']['auto_x'] = filename_x
            json_contents['meta_data']['auto_y'] = filename_y

            

            res = f.read()
            buff = json.dumps(json_contents)
            f.seek(now_pos)
            f.write(buff+'\n')
            now_pos = f.tell()
            f.write(res)
            f.truncate()
            f.seek(now_pos)

In [ ]:


def simple_ui():
    button_clear_output = widgets.Button(description='表示クリア')
    button_apt_device_list = widgets.Button(description='apt_device_list')
    button_stage_enable = widgets.Button(description='stage_enable')
    button_stage_disable = widgets.Button(description='stage_disable')
    button_stage_move_to = widgets.Button(description='stage_move_to')
    button_stage_velocity = widgets.Button(description='stage_velocity')
    button_start_DMM6500 = widgets.Button(description='start_DMM6500')
    button_stop_DMM6500 = widgets.Button(description='stop_DMM6500')
    button_read_DMM6500 = widgets.Button(description='read_DMM6500')
    button_plot_xy = widgets.Button(description='plot_xy')
    button_save_xy = widgets.Button(description='save_xy')
    button_measure_autocorrelation = widgets.Button(description='measure_autocorrelation',layout={'width': 'auto'})
    button_gaussian_fitting_dialog = widgets.Button(description='gaussian_fitting_dialog',layout={'width': 'auto'})
    button_playback_bayesian08 = widgets.Button(description='playback_bayesian08',layout={'width': 'auto'})
    filechooser = FileChooser('./')
    button_load_as_ydata_npy = widgets.Button(description='ydata.npyとして読込')
    button_load_as_xdata = widgets.Button(description='xdata.npyとして読込')
    button_set_as_bayes_json = widgets.Button(description='bayes.jsonとして設定')
    button_input_field = widgets.Button(description='強制変数反映',disabled=False)

    text_comment = widgets.Text(value='',placeholder='文字を入力',description='comment',disabled=False,style={'description_width': 'initial'})
    text_stage_serial_num = widgets.Text(value='28252436',placeholder='28252436',description='stage_serial_num',disabled=False, style={'description_width': 'initial'})
    float_stage_target_pos = widgets.FloatText(value=0,description='stage_target_pos',disabled=False,style={'description_width': 'initial'})
    float_stage_max_velocity = widgets.FloatText(value=99,description='stage_max_velocity',disabled=False,style={'description_width': 'initial'})
    text_DMM6500_visa_address = widgets.Text(value='TCPIP0::169.254.6.100::inst0::INSTR',placeholder='visa_address',description='DMM6500_visa_address',disabled=False, style={'description_width': 'initial'})
    float_auto_stage_start_pos = widgets.FloatText(value=27,description='auto_stage_start_pos',disabled=False,style={'description_width': 'initial'})
    float_auto_stage_end_pos = widgets.FloatText(value=41,description='auto_stage_end_pos',disabled=False,style={'description_width': 'initial'})



    output = widgets.Output()
    def wrapped_func_factory(func,verbose=True):
        def new_func(ui_element):
            with output:
                if verbose: print(f"[exec func {func.__name__}]")
                func()
                if verbose: print(f"[complete {func.__name__}]")
        return new_func
    button_clear_output.on_click(lambda button: output.clear_output(wait=False))

    def load_npy_factory(variable_name):
        def load_npy():
            choosed_file_path = filechooser.selected
            global x,y
            if variable_name == 'y':
                y = np.load(choosed_file_path)
                print(f'y.shape={y.shape}')
            elif variable_name == 'x':
                x = np.load(choosed_file_path)
                print(f'x.shape={x.shape}')
        return load_npy
    
    def set_as_bayes_json():
        global bayes_json
        choosed_file_path = filechooser.selected
        bayes_json = choosed_file_path

    button_load_as_ydata_npy.on_click(wrapped_func_factory(load_npy_factory('y')))
    button_load_as_xdata.on_click(wrapped_func_factory(load_npy_factory('x')))
    button_set_as_bayes_json.on_click(wrapped_func_factory(set_as_bayes_json))
    
    def load_input_field():
        global comment
        global stage_serial_num
        global stage_target_pos
        global stage_max_velocity
        global DMM6500_visa_address
        global auto_stage_start_pos
        global auto_stage_end_pos
        comment = text_comment.value
        stage_serial_num = int(text_stage_serial_num.value)
        stage_target_pos = float_stage_target_pos.value
        stage_max_velocity = float_stage_max_velocity.value
        DMM6500_visa_address = text_DMM6500_visa_address.value
        auto_stage_start_pos = float_auto_stage_start_pos.value
        auto_stage_end_pos = float_auto_stage_end_pos.value
    button_input_field.on_click(wrapped_func_factory(load_input_field,verbose=False))
    button_apt_device_list.on_click(wrapped_func_factory(apt_device_list))
    button_stage_move_to.on_click(wrapped_func_factory(stage_move_to))
    button_stage_velocity.on_click(wrapped_func_factory(stage_velocity))
    button_stage_enable.on_click(wrapped_func_factory(stage_enable))
    button_stage_disable.on_click(wrapped_func_factory(stage_disable))
    button_start_DMM6500.on_click(wrapped_func_factory(start_DMM6500))
    button_stop_DMM6500.on_click(wrapped_func_factory(stop_DMM6500))
    button_read_DMM6500.on_click(wrapped_func_factory(read_DMM6500))
    button_plot_xy.on_click(wrapped_func_factory(plot_xy))
    button_save_xy.on_click(wrapped_func_factory(save_xy))
    button_measure_autocorrelation.on_click(wrapped_func_factory(measure_autocorrelation))
    button_gaussian_fitting_dialog.on_click(wrapped_func_factory(gaussian_fitting_dialog,verbose=False))
    button_playback_bayesian08.on_click(wrapped_func_factory(playback_bayesian08))
    
    input_fields = [text_comment, text_stage_serial_num, float_stage_target_pos, float_stage_max_velocity]
    input_fields+= [text_DMM6500_visa_address]
    input_fields+= [float_auto_stage_end_pos,float_auto_stage_start_pos]
    for field in input_fields:
        field.continuous_update = False
        field.observe(wrapped_func_factory(load_input_field,verbose=False))

    display(
        widgets.VBox([
            widgets.HBox([button_clear_output]),
            widgets.HBox([button_apt_device_list,button_stage_disable, button_stage_enable,button_stage_move_to,button_stage_velocity]),
            widgets.HBox([button_start_DMM6500,button_stop_DMM6500,button_read_DMM6500]),
            widgets.HBox([button_plot_xy, button_save_xy]),
            widgets.HBox([button_measure_autocorrelation, button_gaussian_fitting_dialog]),
            widgets.HBox([filechooser, button_load_as_ydata_npy, button_load_as_xdata, button_set_as_bayes_json]),
            widgets.HBox([text_comment,  button_input_field]),
            widgets.HBox([text_stage_serial_num, float_stage_target_pos,float_stage_max_velocity]),
            widgets.HBox([text_DMM6500_visa_address]),
            widgets.HBox([float_auto_stage_start_pos,float_auto_stage_end_pos]),
            widgets.HBox([button_playback_bayesian08]),
            output
        ])
    )


In [ ]:
simple_ui()